# 🎬 Video & Temporal Vision

**Topics:** Optical Flow, Motion Estimation, Video Analysis

In [ ]:
# Setup
!pip install opencv-python-headless numpy matplotlib -q
import cv2
import numpy as np
import matplotlib.pyplot as plt
print('✅ Setup complete!')

In [ ]:
# Optical Flow Visualization
# Create two synthetic frames with motion
frame1 = np.zeros((200, 200), dtype=np.uint8)
frame2 = np.zeros((200, 200), dtype=np.uint8)

# Circle moving right and down
cv2.circle(frame1, (80, 80), 20, 255, -1)
cv2.circle(frame2, (100, 100), 20, 255, -1)

# Compute Dense Optical Flow (Farneback)
flow = cv2.calcOpticalFlowFarneback(frame1, frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(frame1, cmap='gray')
axes[0].set_title('Frame 1')
axes[1].imshow(frame2, cmap='gray')
axes[1].set_title('Frame 2')

# Convert flow to HSV for visualization
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
hsv = np.zeros((200, 200, 3), dtype=np.uint8)
hsv[..., 0] = ang * 180 / np.pi / 2
hsv[..., 1] = 255
hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
axes[2].imshow(rgb)
axes[2].set_title('Optical Flow')
plt.show()
print('Flow direction: right + down (as expected)')

In [ ]:
# Lucas-Kanade Sparse Optical Flow
# Detect corners in frame1
corners = cv2.goodFeaturesToTrack(frame1, maxCorners=10, qualityLevel=0.3, minDistance=7)

if corners is not None:
    # Track corners to frame2
    next_pts, status, _ = cv2.calcOpticalFlowPyrLK(frame1, frame2, corners, None)
    
    # Draw tracks
    img = cv2.cvtColor(frame2, cv2.COLOR_GRAY2BGR)
    for i, (new, old) in enumerate(zip(next_pts, corners)):
        if status[i]:
            a, b = new.ravel().astype(int)
            c, d = old.ravel().astype(int)
            cv2.line(img, (c, d), (a, b), (0, 255, 0), 2)
            cv2.circle(img, (a, b), 3, (0, 0, 255), -1)
    
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Lucas-Kanade Tracking')
    plt.show()
else:
    print('No corners detected')